# Building pipelines that perform routing



## Conditional routing

This example will route the query to different processing paths, such as checking the query length and checking if the query contains specific keywords.

Scenario: Keyword Detection Routing:

* If the query contains the keyword "capital", it routes to a query generation component that fetches city names.
* If the query doesn't contain "capital", it goes to a general information retrieval system that can provide a broader response.

We'll build this example using a conditional router that handles both conditions based on query length and keyword detection.

In [4]:
from haystack import Pipeline
from haystack.components.routers import ConditionalRouter
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.components.generators import OpenAIGenerator
from dotenv import load_dotenv
import os
from haystack.utils import Secret

load_dotenv('.env')

openai_api_key = os.getenv("OPENAI_API_KEY")

# Example of a pipeline with a ConditionalRouter that routes 
# queries based on their keyword presence

# Define the routes based on query length and keyword check
routes = [

    {
        "condition": "{{'capital' in query}}",  # Check if the query contains the keyword "capital"
        "output": "{{query}}",  # Proceed with the query if "capital" is in the query
        "output_name": "capital_related_query",
        "output_type": str,
    },
    {
        "condition": "{{'capital' not in query}}",  # Otherwise, handle general queries
        "output": "This is a general query: {{query}}",
        "output_name": "general_query",
        "output_type": str,
    }
]

# Create the router
router = ConditionalRouter(routes=routes)

# Create the pipeline with components
pipe = Pipeline()

# Add the router, prompt builder, document retriever, and generator
pipe.add_component("router", router)
pipe.add_component("prompt_builder", PromptBuilder("Answer the following query: {{query}}"))
pipe.add_component("generator", OpenAIGenerator( api_key= Secret.from_env_var("OPENAI_API_KEY"),
))

# Connect the components
pipe.connect("router.capital_related_query", "prompt_builder.query")
pipe.connect("prompt_builder", "generator")

# Example 1: A short query that triggers a warning
result_short = pipe.run(data={"router": {"query": "Berlin"}})
print(result_short)
# Expected output: {'router': {'short_query_warning': 'Query is too short: Berlin'}}

# Example 2: A longer query containing the keyword "capital"
result_long_capital = pipe.run(data={"router": {"query": "What is the capital of France?"}})
print(result_long_capital)
# Expected output: {'generator': {'replies': ['The capital of France is Paris.']}}

# Example 3: A longer query without the keyword "capital"
result_long_general = pipe.run(data={"router": {"query": "Tell me about the Eiffel Tower."}})
print(result_long_general)
# Expected output: {'generator': {'replies': ['The Eiffel Tower is a famous landmark in Paris.']}}


{'router': {'general_query': 'This is a general query: Berlin'}}
{'generator': {'replies': ['The capital of France is Paris.'], 'meta': [{'model': 'gpt-4o-mini-2024-07-18', 'index': 0, 'finish_reason': 'stop', 'usage': {'completion_tokens': 7, 'prompt_tokens': 19, 'total_tokens': 26, 'completion_tokens_details': CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), 'prompt_tokens_details': PromptTokensDetails(audio_tokens=0, cached_tokens=0)}}]}}
{'router': {'general_query': 'This is a general query: Tell me about the Eiffel Tower.'}}


## Routing based on file type

This router is useful for routing different file types (e.g., plain text, PDF, images, audio) to different components based on their MIME types. In this example, we will:

Route file paths:
Plain text files will be processed using a converter and splitter, while PDF files will be routed separately and converted into documents using a PDF converter.
Unclassified files (files that don't match any MIME types provided) will be handled as an "unclassified" category.

Scenario:
We have the following files:

* A plain text file: example.txt
* An image file: image.jpg


The objective is to:

* Convert the plain text file into a document.
* Skip the image file as it is unclassified and doesn't match any of the specified MIME types.

We will use the `FileTypeRouter` to route these files to their respective processing paths.

In [8]:
from haystack import Pipeline
from haystack.components.routers import FileTypeRouter
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.converters import TextFileToDocument
from haystack.components.preprocessors import DocumentSplitter
from haystack.components.writers import DocumentWriter

# Create an in-memory document store to hold the processed documents
document_store = InMemoryDocumentStore()

# Initialize the pipeline
pipeline = Pipeline()

# Add the FileTypeRouter that routes only 'text/plain' and 'application/pdf'
pipeline.add_component(instance=FileTypeRouter(mime_types=["text/plain", "application/pdf"]), name="file_type_router")

# Add components for text file conversion and PDF conversion
pipeline.add_component(instance=TextFileToDocument(), name="text_file_converter")

# Add components for splitting and writing documents
pipeline.add_component(instance=DocumentSplitter(), name="splitter")
pipeline.add_component(instance=DocumentWriter(document_store=document_store), name="writer")

# Connect components in the pipeline
pipeline.connect("file_type_router.text/plain", "text_file_converter.sources")
pipeline.connect("text_file_converter.documents", "splitter.documents")
pipeline.connect("splitter.documents", "writer.documents")

# Run the pipeline with a list of file paths
result = pipeline.run({"file_type_router": {"sources": ["example.txt", "image.jpeg"]}})

print(result)

{'file_type_router': {'unclassified': [PosixPath('image.jpeg')]}, 'writer': {'documents_written': 1}}


More on routers

https://docs.haystack.deepset.ai/docs/routers